# Network Creation and Model Simulation

Perform computationally expensive network creation, some network analysis,
and the model simulation. Results are pushed to a data repository on GitHub.

- Create networks and run network metric calculations
- Create model configurations and run simulations
- Run simulations for model validation

In [1]:
# to allow relative imports
import os
from sys_path_util import append_sys_path
append_sys_path()

# experiment
from epyc import Lab, JSONLabNotebook, RepeatedExperiment, ParallelLab
from lib.experiments.utils.metrics import MetricExperiment, calc_shortest_paths

# generators
from epydemic import PLCNetwork
from lib.model.network.mobility_network import MNGeneratorFromNetworkData as MNG
from lib.model.network.distanced_network import DNGenerator as DNG

# distributions
from lib.model.distributions import discrete_trunc_normal, discrete_trunc_exponential, num_contact_dist

# file utils
from lib.experiments.utils.data_repo_api import DataRepoAPI
from lib.experiments.utils.network_data_utils import load_network_data_from_files

# other utils
from functools import partial
import numpy as np
from mpmath import polylog
from datetime import datetime as dt
import pickle
import lzma

# multiprocessing
from joblib import Parallel, delayed
from multiprocessing import cpu_count

# Network creation and analysis

Create the Mobility, PLC, and Distanced networks and calculate the
network metrics as described in the thesis.

In [2]:
# Some global parameters

N = 10000
CUTOFF = 40
MOBILITY_EXPONENT = 2
n_exp = 10

# Experiment result output dir
output_dir = 'experiment_results'
if not output_dir in os.listdir():
    os.mkdir(output_dir)

# Helper functions
def run_network_lab_experiment(generator, params, file_name, description, n=n_exp,
                               output_dir=output_dir, push_to_repo=True):
    """
    Run a MetricExperiment repeatedly using a network generator.
    """

    print(f"%s - Running {file_name}" % dt.now())
    
    file = os.path.join(output_dir, file_name)
    
    nb = JSONLabNotebook(file, create=True, description=description)
    lab = ParallelLab(notebook=nb, cores=min(cpu_count()-1, n))
    
    e = MetricExperiment(generator)

    # pass params to the lab
    for k, v in params.items():
        lab[k] = v

    lab.runExperiment(RepeatedExperiment(e, n))
    
    print("%s - Done." % dt.now())

    if push_to_repo:
        DataRepoAPI.update_or_create(
            file_name=file_name,
            file_path=output_dir,
            repo_path='network-metrics'
        )

    return lab.results()

def plc_mean(exponent, cutoff):
    """
    Calculate theoretical mean of a power law with cutoff distribution from
    the exponent and the cutoff.
    """
    nom = polylog(exponent-1, np.exp(-1 / cutoff))
    denom = polylog(exponent, np.exp(-1 / cutoff))
    return nom / denom

def estimate_exponent_for_mean(mean, cutoff, tolerance):
    """
    Given the mean and cutoff of a power law with cutoff distribution,
    estimate the corresponding exponent with a given tolerance.
    """
    grid = np.arange(1, 5, 0.01)
    diffs = []
    for i in range(len(grid)):
        diff = abs(mean - plc_mean(grid[i], cutoff))
        if diff < tolerance:
            return grid[i]
        diffs.append(diff)
    
    return grid[diffs.index(min(diffs))]

### Mobility network

In [3]:
# Load the network data

file_names = dict(
    demographics='demographics.pkl',
    comb_pre='comb_counts_pre.pkl',
    comb_post='comb_counts_post.pkl',
    trip_pre='trip_counts_pre.pkl',
    trip_post='trip_counts_post.pkl',
)

network_data = load_network_data_from_files(file_names)

network_data_pre = network_data['pre']
network_data_post = network_data['post']

In [4]:
# Define the generator parameters
params_mobility_pre = dict()
params_mobility_pre[MNG.N] = N
params_mobility_pre[MNG.EXPONENT] = MOBILITY_EXPONENT
params_mobility_pre[MNG.CUTOFF] = CUTOFF
params_mobility_pre[MNG.MULTIPLIER] = False

params_mobility_post = dict()
params_mobility_post[MNG.N] = N
params_mobility_post[MNG.EXPONENT] = MOBILITY_EXPONENT
params_mobility_post[MNG.CUTOFF] = CUTOFF
params_mobility_post[MNG.MULTIPLIER] = True

In [5]:
# Generator instances
mng_pre = MNG(network_data=network_data_pre)
mng_post = MNG(network_data=network_data_post)

In [ ]:
# Run experiment
mng_pre_results = run_network_lab_experiment(
    mng_pre, 
    params_mobility_pre, 
    'mobility_pre.json', 
    'Network metrics of Mobility pre'
)


mng_post_results = run_network_lab_experiment(
    mng_post, 
    params_mobility_post, 
    'mobility_post.json', 
    'Network metrics of Mobility post'
)

In [6]:
def extract_mean_deg(results):
    deg_lis = [x['results']['degrees'] for x in results]
    return np.mean([i for s in deg_lis for i in s])

# mob_deg_pre = extract_mean_deg(mng_pre_results)
# mob_deg_post = extract_mean_deg(mng_post_results)

In [7]:
# get from repo...
mob_pre = DataRepoAPI.get_json_file('network-metrics/mobility_pre.json')
mob_post = DataRepoAPI.get_json_file('network-metrics/mobility_post.json')

_results_pre = mob_pre['resultsets']['epyc.resultset.default']['results']
_results_post = mob_post['resultsets']['epyc.resultset.default']['results']

mob_deg_pre = extract_mean_deg(_results_pre)
mob_deg_post = extract_mean_deg(_results_post)

print(mob_deg_pre, mob_deg_post)

4.185571248201544 3.1256872297548473


### PLC network

We needed the results of the mng for calculating the exponent of the other experiments..
but the rest we can run in parallel to save some time...

In [8]:
# List to store all the metric experiment runners
mtrcs = list()

In [8]:
# Estimate the exponent for the PLC to achieve same degree
#  as the mobility networks

plc_expo_pre = estimate_exponent_for_mean(mob_deg_pre, CUTOFF, 0.01)
plc_expo_post = estimate_exponent_for_mean(mob_deg_post, CUTOFF, 0.01)

In [9]:
# Define the generator parameters
params_plc_pre = dict()
params_plc_pre[PLCNetwork.N] = N
params_plc_pre[PLCNetwork.EXPONENT] = plc_expo_pre
params_plc_pre[PLCNetwork.CUTOFF] = CUTOFF

params_plc_post = dict()
params_plc_post[PLCNetwork.N] = N
params_plc_post[PLCNetwork.EXPONENT] = plc_expo_post
params_plc_post[PLCNetwork.CUTOFF] = CUTOFF

In [9]:
plc_pre = PLCNetwork()
plc_post = PLCNetwork()

In [ ]:
mtrcs.append(lambda: run_network_lab_experiment(
    plc_pre, 
    params_plc_pre, 
    'plc_pre.json', 
    'Network metrics of PLC pre'
))

mtrcs.append(lambda: run_network_lab_experiment(
    plc_post, 
    params_plc_post, 
    'plc_post.json', 
    'Network metrics of PLC post'
))

### Distanced network

In [10]:
household_size_dist = partial(discrete_trunc_normal, mu=4.5, std=2)

params_distanced_pre = dict()
params_distanced_pre[DNG.N] = N

params_distanced_post = dict()
params_distanced_post[DNG.N] = N

In [11]:
distanced_pre = DNG(
    household_size_dist=household_size_dist,
    num_contact_dist=num_contact_dist,
    num_outside_edge_dist=partial(discrete_trunc_exponential, exponent=mob_deg_pre)
)

distanced_post = DNG(
    household_size_dist=household_size_dist,
    num_contact_dist=num_contact_dist,
    num_outside_edge_dist=partial(discrete_trunc_exponential, exponent=mob_deg_post)
)

In [15]:
mtrcs.append(lambda: run_network_lab_experiment(
    distanced_pre, 
    params_distanced_pre, 
    'distanced_pre.json', 
    'Network metrics of PLC pre'
))

mtrcs.append(lambda: run_network_lab_experiment(
    distanced_post, 
    params_distanced_post, 
    'distanced_post.json', 
    'Network metrics of PLC post'
))

In [22]:
# # Alternative way of running if epyc.Lab is used
# num_cores = min(len(mtrcs), cpu_count()-1)
# with Parallel(n_jobs=num_cores) as processes:
#     procs = processes(delayed(e)() for e in mtrcs)

for mtrc in mtrcs:
    print("Start time: %s" % dt.now())
    mtrc()
    print("End time: %s" % dt.now())

Done
Done
Done
Done
CPU times: user 25.9 ms, sys: 5.08 ms, total: 31 ms
Wall time: 3min 42s


### Shortest path lengths

For the network analysis, we calculate the shortest path lengths for one instance
of each network type.

In [40]:
def run_sp_calc(generator, params, file_name):
    generator.set(params)
    g = generator.generate()
    sp = calc_shortest_paths(g)

    return (sp, file_name)

In [41]:
sp_calc_params = [
    (mng_pre, params_mobility_pre, 'sp_mng_pre'),
    (mng_post, params_mobility_post, 'sp_mng_post'),
    (plc_pre, params_plc_pre, 'sp_plc_pre'),
    (plc_post, params_plc_post, 'sp_plc_post'),
    (distanced_pre, params_distanced_pre, 'sp_dist_pre'),
    (distanced_post, params_distanced_post, 'sp_dist_post')
]

In [43]:
# Run this using multiprocessing
num_cores = min(len(sp_calc_params), cpu_count()-1, 6)
print(f"Running {num_cores} jobs on {cpu_count()} available cores...")

print("Start time %s" % dt.now())

with Parallel(n_jobs=num_cores) as processes:
    procs = processes(delayed(lambda spc: run_sp_calc(*spc))(spc) for spc in sp_calc_params)

    print('Done with calculations...')

    for sp, file_name in procs:
        fn = file_name + '.pkl'

        with lzma.open(os.path.join(output_dir, fn), 'wb') as f:
            pickle.dump(sp, f)

        DataRepoAPI.update_or_create(
            file_name=fn,
            file_path=output_dir,
            repo_path='network-metrics'
        )

        print(f'Done {fn}')
    
print("End time %s" % dt.now())

Running 6 jobs on 12 available cores...
Start time 2021-07-22 07:22:48.817079
End time 2021-07-22 07:22:50.010002


# Epidemic models

In this part, we will create the different configurations of epidemic models
and run the simulations on the networks.

In [13]:
# List to store all the experiment runners
expms = list()

In [12]:
# Import models etc
from epydemic import SEIR
from lib.model.compartmental_model.seir import MonitoredSEIR
from lib.model.compartmental_model.seir import SEIRWithQuarantine as SEIRQ
from lib.model.compartmental_model.seir import MonitoredSEIRWithQuarantine as MonitoredSEIRQ
from lib.model.compartmental_model.seivr import SEIVR, MonitoredSEIVR
from lib.model.compartmental_model.seivr import SEIVRWithQuarantine as SEIVRQ
from lib.model.compartmental_model.seivr import MonitoredSEIVRWithQuarantine as MonitoredSEIVRQ

from epydemic import StochasticDynamics, Monitor

from lib.model.distributions import PowerLawCutoffDist as PLCD

In [13]:
# Helper functions

def calc_p_infect(r0, p_remove, k_mean, k_var):
    """
    Calc p_infect.
    :param r0: basic reproduction number
    :param p_remove: probability of removal
    :param k_mean: Mean of node degree distribution
    :param k_var: Variance of node degree distribution
    :return: p_infect
    """
    return float(p_remove * r0 * (k_mean / (k_var - k_mean)))


def run_stochastic_dynamics_lab(model, generator, params, 
                                file_name, description, n=n_exp,
                                output_dir=output_dir, push_to_repo=True,
                                T_max=300):
    """
    Run a StochasticDynamics experiment repeatadly in a lab.
    """

    file = os.path.join(output_dir, file_name)
    
    nb = JSONLabNotebook(file, create=True, description=description)
    
    cores = min(cpu_count()-1, n)
    print(f"Cores: {cores}")
    lab = ParallelLab(notebook=nb, cores=cores)
    
    e = StochasticDynamics(model, generator)
    e.process().setMaximumTime(T_max)

    # pass params to the lab
    for k, v in params.items():
        lab[k] = v

    lab.runExperiment(RepeatedExperiment(e, n))
    
    print(f"Experiment {file_name} done: %s" % dt.now())
    
    if push_to_repo:
        DataRepoAPI.update_or_create(
            file_name=file_name,
            file_path=output_dir,
            repo_path='simulations'
        )
    
    return lab.results()

## Parameters

We will fix as many COVID-19 specific parameters as we can and leave only policy
dependent parameters open for modifications. References for each of the parameters
are explained in the thesis.

In [14]:
# Fixed
R0 = 2.85
_P_INFECT = None  # to calculate later 
P_REMOVE = 0.1
P_SYMPTOMS = 0.2
P_EXPOSED = 0.01

_RRR = [0.5, 0.65, 0.8, 0.95]

# Free
_P_VACCINATED = [0.001, 0.004, 0.007, 0.01]
_P_VACCINATED_INITAL = [0.0, 0.2, 0.4, 0.6, 0.8]
_P_QUARANTINE = [0.0, 0.25, 0.5, 0.75, 1]

In [15]:
# Fix parameters as far as possible

params_seir = dict()
params_seir[SEIR.P_SYMPTOMS] = P_SYMPTOMS
params_seir[SEIR.P_REMOVE] = P_REMOVE
params_seir[SEIR.P_EXPOSED] = P_EXPOSED

params_seivr = dict()
params_seivr[SEIVR.P_SYMPTOMS] = P_SYMPTOMS
params_seivr[SEIVR.P_REMOVE] = P_REMOVE
params_seivr[SEIVR.P_EXPOSED] = P_EXPOSED

params_monitor = dict()
params_monitor[Monitor.DELTA] = 10

In [16]:
# Calculate P_INFECT

# Mobility
plcd = PLCD(MOBILITY_EXPONENT, CUTOFF)
P_INFECT_MOBILITY = calc_p_infect(R0, P_REMOVE, plcd.mean, plcd.var)

# PLC
plcd = PLCD(plc_expo_pre, CUTOFF)
P_INFECT_PLC = calc_p_infect(R0, P_REMOVE, plcd.mean, plcd.var)

# Distanced
mean = mob_deg_pre
var = mean ** 2
P_INFECT_DISTANCED = calc_p_infect(R0, P_REMOVE, mean, var)

# clean up
del mean, var, plcd

print(P_INFECT_MOBILITY, P_INFECT_PLC, P_INFECT_DISTANCED)

0.03931425340497793 0.018206599811410505 0.089465900397268


### SEIR & SEIR_Q

#### SEIR on Mobility

In [146]:
# params
params_pre1 = {**params_mobility_pre, **params_seir, **params_monitor}
params_pre1[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_pre1[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2

params_post1 = {**params_mobility_post, **params_seir, **params_monitor}
params_post1[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_post1[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2

In [147]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIR(),
    mng_pre, 
    params_pre1, 
    'seir_mobility_pre.json',
    'SEIR simulation with Mobility Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIR(), 
    mng_post, 
    params_post1, 
    'seir_mobility_post.json',
    'SEIR simulation with Mobility Network (Post)'
))

#### SEIR_Q on Mobility

In [148]:
params_pre2 = {**params_mobility_pre, **params_seir, **params_monitor}
params_pre2[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_pre2[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_pre2[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

params_post2 = {**params_mobility_post, **params_seir, **params_monitor}
params_post2[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_post2[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_post2[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

In [149]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    mng_pre, 
    params_pre2, 
    'seirq_mobility_pre.json',
    'SEIR_Q simulation with Mobility Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    mng_post, 
    params_post2, 
    'seirq_mobility_post.json',
    'SEIR_Q simulation with Mobility Network (Post)'
))

#### SEIR on PLC

In [19]:
params_pre3 = {**params_plc_pre, **params_seir, **params_monitor}
params_pre3[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_pre3[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2

params_post3 = {**params_plc_post, **params_seir, **params_monitor}
params_post3[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_post3[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2

In [21]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIR(), 
    PLCNetwork(), 
    params_pre3, 
    'seir_plc_pre.json',
    'SEIR simulation with PLC Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIR(), 
    PLCNetwork(), 
    params_post3, 
    'seir_plc_post.json',
    'SEIR simulation with PLC Network (Post)'
))

#### SEIR_Q on PLC

In [20]:
params_pre4 = {**params_plc_pre, **params_seir, **params_monitor}
params_pre4[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_pre4[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2
params_pre4[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

params_post4 = {**params_plc_post, **params_seir, **params_monitor}
params_post4[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_post4[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2
params_post4[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

In [21]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    PLCNetwork(), 
    params_pre4, 
    'seirq_plc_pre.json',
    'SEIR_Q simulation with PLC Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    PLCNetwork(), 
    params_post4, 
    'seirq_plc_post.json',
    'SEIR_Q simulation with PLC Network (Post)'
))

#### SEIR on Distanced

In [24]:
params_pre5 = {**params_distanced_pre, **params_seir, **params_monitor}
params_pre5[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_pre5[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2

params_post5 = {**params_distanced_post, **params_seir, **params_monitor}
params_post5[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_post5[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2

In [25]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIR(), 
    distanced_pre, 
    params_pre5, 
    'seir_distanced_pre.json',
    'SEIR simulation with Distanced Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIR(), 
    distanced_post, 
    params_post5, 
    'seir_distanced_post.json',
    'SEIR simulation with Distanced Network (Post)'
))

#### SEIR_Q on Distanced

In [29]:
params_pre6 = {**params_distanced_pre, **params_seir, **params_monitor}
params_pre6[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_pre6[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2
params_pre6[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

params_post6 = {**params_distanced_post, **params_seir, **params_monitor}
params_post6[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_post6[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2
params_post6[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

In [30]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    distanced_pre, 
    params_pre6, 
    'seirq_distanced_pre.json',
    'SEIR_Q simulation with Distanced Network (Pre)',
))

In [31]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    distanced_post, 
    params_post6, 
    'seirq_distanced_post.json',
    'SEIR_Q simulation with Distanced Network (Post)'
))

### SEIVR & SEIVR_Q

#### SEIVR on Mobility

In [260]:
params_pre7 = {**params_mobility_pre, **params_seivr, **params_monitor}
params_pre7[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_pre7[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_pre7[SEIVR.P_VACCINATED] = _P_VACCINATED
params_pre7[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre7[SEIVR.VACCINE_RRR] = _RRR

params_post7 = {**params_mobility_post, **params_seivr, **params_monitor}
params_post7[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_post7[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_post7[SEIVR.P_VACCINATED] = _P_VACCINATED
params_post7[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post7[SEIVR.VACCINE_RRR] = _RRR

In [261]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    mng_pre, 
    params_pre7, 
    'seivr_mobility_pre.json',
    'SEIVR simulation with Mobility Network (Pre)',
))

In [ ]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    mng_post, 
    params_post7, 
    'seivr_mobility_post.json',
    'SEIVR simulation with Mobility Network (Post)'
))

#### SEIVR_Q on Mobility

In [246]:
params_pre8 = {**params_mobility_pre, **params_seivr, **params_monitor}
params_pre8[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_pre8[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_pre8[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_pre8[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre8[SEIVRQ.VACCINE_RRR] = _RRR
params_pre8[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

params_post8 = {**params_mobility_post, **params_seivr, **params_monitor}
params_post8[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_post8[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_post8[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_post8[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post8[SEIVRQ.VACCINE_RRR] = _RRR
params_post8[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

In [247]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    mng_pre, 
    params_pre8, 
    'seivrq_mobility_pre.json',
    'SEIVR_Q simulation with Mobility Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    mng_post, 
    params_post8, 
    'seivrq_mobility_post.json',
    'SEIVR_Q simulation with Mobility Network (Post)'
))

#### SEIVR on PLC

In [20]:
params_pre9 = {**params_plc_pre, **params_seivr, **params_monitor}
params_pre9[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_pre9[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2
params_pre9[SEIVR.P_VACCINATED] = _P_VACCINATED
params_pre9[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre9[SEIVR.VACCINE_RRR] = _RRR

params_post9 = {**params_plc_post, **params_seivr, **params_monitor}
params_post9[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_post9[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2
params_post9[SEIVR.P_VACCINATED] = _P_VACCINATED
params_post9[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post9[SEIVR.VACCINE_RRR] = _RRR

In [21]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    PLCNetwork(), 
    params_pre9, 
    'seivr_plc_pre.json',
    'SEIVR simulation with PLC Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    PLCNetwork(), 
    params_post9, 
    'seivr_plc_post.json',
    'SEIVR simulation with PLC Network (Post)'
))

#### SEIVR_Q on PLC

In [19]:
params_pre10 = {**params_plc_pre, **params_seivr, **params_monitor}
params_pre10[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_pre10[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2
params_pre10[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_pre10[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre10[SEIVRQ.VACCINE_RRR] = _RRR
params_pre10[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

params_post10 = {**params_plc_post, **params_seivr, **params_monitor}
params_post10[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_post10[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2
params_post10[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_post10[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post10[SEIVRQ.VACCINE_RRR] = _RRR
params_post10[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

In [20]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    PLCNetwork(), 
    params_pre10, 
    'seivrq_plc_pre.json',
    'SEIVR_Q simulation with PLC Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    PLCNetwork(), 
    params_post10, 
    'seivrq_plc_post.json',
    'SEIVR_Q simulation with PLC Network (Post)'
))

#### SEIVR on Distanced

In [22]:
params_pre11 = {**params_distanced_pre, **params_seivr, **params_monitor}
params_pre11[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_pre11[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2
params_pre11[SEIVR.P_VACCINATED] = _P_VACCINATED
params_pre11[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre11[SEIVR.VACCINE_RRR] = _RRR

params_post11 = {**params_distanced_post, **params_seivr, **params_monitor}
params_post11[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_post11[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2
params_post11[SEIVR.P_VACCINATED] = _P_VACCINATED
params_post11[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post11[SEIVR.VACCINE_RRR] = _RRR

In [23]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    distanced_pre, 
    params_pre11, 
    'seivr_distanced_pre.json',
    'SEIVR simulation with Distanced Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    distanced_post, 
    params_post11, 
    'seivr_distanced_post.json',
    'SEIVR simulation with Distanced Network (Post)'
))

#### SEIVR_Q on Distanced

In [20]:
params_pre12 = {**params_distanced_pre, **params_seivr, **params_monitor}
params_pre12[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_pre12[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2
params_pre12[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_pre12[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre12[SEIVRQ.VACCINE_RRR] = _RRR
params_pre12[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

params_post12 = {**params_distanced_post, **params_seivr, **params_monitor}
params_post12[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_post12[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2
params_post12[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_post12[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post12[SEIVRQ.VACCINE_RRR] = _RRR
params_post12[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

In [21]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    distanced_pre, 
    params_pre12, 
    'seivrq_distanced_pre.json',
    'SEIVR_Q simulation with Distanced Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    distanced_post, 
    params_post12, 
    'seivrq_distanced_post.json',
    'SEIVR_Q simulation with Distanced Network (Post)'
))

## Run experiments with multiprocessing...


In [32]:
print(len(expms))

2


In [33]:
# # Alternative way of running in case epyc.Lab is used...
# num_cores = min(len(expms), cpu_count()-1, 12)
# print(f"Running {num_cores} jobs on {cpu_count()} available cores...")
# print("Start time %s" % dt.now())
# with Parallel(n_jobs=num_cores) as processes:
#     procs = processes(delayed(e)() for e in expms)
# print("End time %s" % dt.now())

for expm in expms:
    print("Start time: %s" % dt.now())
    expm()
    print("End time: %s" % dt.now())

Start time: 2021-07-29 20:45:06.612411
Cores: 10
Experiment seirq_distanced_pre.json done: 2021-07-29 20:46:36.297905
End time: 2021-07-29 20:46:37.482842
Start time: 2021-07-29 20:46:37.482909
Cores: 10
Experiment seirq_distanced_post.json done: 2021-07-29 20:47:51.096084
End time: 2021-07-29 20:47:52.210660


Once the simulations are complete, transform the simulation files into the format required by
the web app and store them in the data repo.

Alternatively, run `make create_app_data` from the project root - it calls the same function...

In [ ]:
# transform to format required and upload to data repo
from lib.experiments.utils.create_app_data import main
main()

# Model Validation

In this section we run the simulations used for validation.

In [25]:
n_exp_v = 10

def run_validation_lab(model, generator, params, file_name, description, n=n_exp_v,
                       output_dir=output_dir, push_to_repo=True, T_max=200):
    """
    Run a StochasticDynamics experiment repeatedly in a lab.
    """
    
    print(file_name)

    file = os.path.join(output_dir, file_name)
    
    nb = JSONLabNotebook(file, create=True, description=description)
    lab = ParallelLab(notebook=nb, cores=min(cpu_count()-1, n))
    
    e = StochasticDynamics(model, generator)
    e.process().setMaximumTime(T_max)

    # pass params to the lab
    for k, v in params.items():
        lab[k] = v

    lab.runExperiment(RepeatedExperiment(e, n))
    
    print(f"Experiment {file_name} done: %s" % dt.now())
    
    if push_to_repo:
        DataRepoAPI.update_or_create(
            file_name=file_name,
            file_path=output_dir,
            repo_path='validation'
        )
    
    return lab.results()

In [43]:
validations = []
v_params_monitor = dict()
v_params_monitor[Monitor.DELTA] = 1

In [30]:
# First wave
first_v_params_seir = dict()
first_v_params_seir[SEIR.P_SYMPTOMS] = P_SYMPTOMS
first_v_params_seir[SEIR.P_REMOVE] = P_REMOVE
first_v_params_seir[SEIR.P_EXPOSED] = 0.001

### First wave SEIR (all networks)

In [31]:
seir_v_params = {
    **first_v_params_seir, **v_params_monitor,
    SEIR.P_INFECT_SYMPTOMATIC: P_INFECT_MOBILITY,
    SEIR.P_INFECT_ASYMPTOMATIC: P_INFECT_MOBILITY / 2
},

#### SEIR on Mobility

In [76]:
# params

validations.append(lambda: run_validation_lab(
    MonitoredSEIR(),
    mng_pre, 
    {
        **params_mobility_pre, 
        **seir_v_params
    },
    'v_seir_mobility_pre.json',
    'SEIR simulation with Mobility Network (Pre)',
))


validations.append(lambda: run_validation_lab(
    MonitoredSEIR(), 
    mng_post, 
    {
        **params_mobility_post, 
        **seir_v_params
    },
    'v_seir_mobility_post.json',
    'SEIR simulation with Mobility Network (Post)'
))

#### SEIR on PLC

In [77]:
validations.append(lambda: run_validation_lab(
    MonitoredSEIR(), 
    PLCNetwork(), 
    {
        **params_plc_pre,
        **seir_v_params
    }, 
    'v_seir_plc_pre.json',
    'SEIR simulation with PLC Network (Pre)',
))


validations.append(lambda: run_validation_lab(
    MonitoredSEIR(), 
    PLCNetwork(), 
    {
        **params_plc_post,         
        **seir_v_params
    }, 
    'v_seir_plc_post.json',
    'SEIR simulation with PLC Network (Post)'
))

#### SEIR on Distanced

In [78]:
validations.append(lambda: run_validation_lab(
    MonitoredSEIR(), 
    distanced_pre, 
    {
        **params_distanced_pre,
        **seir_v_params
    }, 
    'v_seir_distanced_pre.json',
    'SEIR simulation with Distanced Network (Pre)',
))


validations.append(lambda: run_validation_lab(
    MonitoredSEIR(), 
    distanced_post, 
    {
        **params_distanced_post, 
        **seir_v_params
    }, 
    'v_seir_distanced_post.json',
    'SEIR simulation with Distanced Network (Post)'
))

### First wave SEIR_Q with Mobility Pre

In [32]:
seirq_mob_pre_params = {
    **params_mobility_pre, **first_v_params_seir, **v_params_monitor,
    SEIRQ.P_INFECT_SYMPTOMATIC: P_INFECT_MOBILITY,
    SEIRQ.P_INFECT_ASYMPTOMATIC: P_INFECT_MOBILITY / 2,
}

In [33]:
# params

validations.append(lambda: run_validation_lab(
    MonitoredSEIRQ(), 
    mng_pre, 
    {
        **seirq_mob_pre_params,
        SEIRQ.P_QUARANTINE: 0.25
    },
    'v_seirq_25_mobility_pre.json',
    'SEIR_Q simulation with Mobility Network (Pre)'
))

validations.append(lambda: run_validation_lab(
    MonitoredSEIRQ(), 
    mng_pre, 
    {
        **seirq_mob_pre_params,
        SEIRQ.P_QUARANTINE: 0.5
    },
    'v_seirq_50_mobility_pre.json',
    'SEIR_Q simulation with Mobility Network (Pre)'
))

validations.append(lambda: run_validation_lab(
    MonitoredSEIRQ(), 
    mng_pre, 
    {
        **seirq_mob_pre_params,
        SEIRQ.P_QUARANTINE: 0.75
    },
    'v_seirq_75_mobility_pre.json',
    'SEIR_Q simulation with Mobility Network (Pre)'
))

### First wave SEIR_Q with Mobility Post

In [ ]:
seirq_mob_post_params = {
    **params_mobility_post, **first_v_params_seir, **v_params_monitor,
    SEIRQ.P_INFECT_SYMPTOMATIC: P_INFECT_MOBILITY,
    SEIRQ.P_INFECT_ASYMPTOMATIC: P_INFECT_MOBILITY / 2,
}

In [79]:
# params

validations.append(lambda: run_validation_lab(
    MonitoredSEIRQ(), 
    mng_post, 
    {
        **seirq_mob_post_params,
        SEIRQ.P_QUARANTINE: 0.25
    },
    'v_seirq_25_mobility_post.json',
    'SEIR_Q simulation with Mobility Network (Post)'
))

validations.append(lambda: run_validation_lab(
    MonitoredSEIRQ(), 
    mng_post, 
    {
        **seirq_mob_post_params,
        SEIRQ.P_QUARANTINE: 0.5
    },
    'v_seirq_50_mobility_post.json',
    'SEIR_Q simulation with Mobility Network (Post)'
))

validations.append(lambda: run_validation_lab(
    MonitoredSEIRQ(), 
    mng_post, 
    {
        **seirq_mob_post_params,
        SEIRQ.P_QUARANTINE: 0.75
    },
    'v_seirq_75_mobility_post.json',
    'SEIR_Q simulation with Mobility Network (Post)'
))

### Vaccination

In [39]:
# parameterised for Connecticut at 01/01/2021

# References:
# (1) https://www.worldometers.info/coronavirus/usa/connecticut/
# (2) https://coronavirus.jhu.edu/region/us/connecticut

# ... more details in the thesis.

v_params_seivr = dict()
v_params_seivr[SEIVR.P_SYMPTOMS] = P_SYMPTOMS
v_params_seivr[SEIVR.P_REMOVE] = P_REMOVE
v_params_seivr[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
v_params_seivr[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2

v_params_seivr[SEIVR.P_REMOVED_INITIAL] = 0.0316  # total cases - active cases; (1)
v_params_seivr[SEIVR.P_EXPOSED] = 0.0068  # 33% of active cases; (1)
v_params_seivr[SEIVR.P_INFECTED_INITIAL] = 0.0137  # 67% of active cases; (1)
v_params_seivr[SEIVR.P_VACCINATED_INITIAL] = 0.0154  # (2)
v_params_seivr[SEIVR.P_VACCINATED] = 0.0091  # (2)
v_params_seivr[SEIVR.VACCINE_RRR] = 0.5  # (2)

### SEIVR with Mobility

In [40]:
validations.append(lambda: run_validation_lab(
    MonitoredSEIVR(),
    mng_pre, 
    {
        **params_mobility_pre, 
        **v_params_seivr,
        **v_params_monitor
    },
    'v_seivr_mobility_pre.json',
    'SEIVR simulation with Mobility Network (Pre)',
))

validations.append(lambda: run_validation_lab(
    MonitoredSEIVR(),
    mng_post, 
    {
        **params_mobility_post, 
        **v_params_seivr,
        **v_params_monitor
    },
    'v_seivr_mobility_post.json',
    'SEIVR simulation with Mobility Network (Post)',
))

### SEIVR_Q with Mobility

#### Pre

In [44]:
validations.append(lambda: run_validation_lab(
    MonitoredSEIVRQ(),
    mng_pre, 
    {
        **params_mobility_pre, 
        **v_params_seivr,
        **v_params_monitor,
        SEIVRQ.P_QUARANTINE: 0.25
    },
    'v_seivrq_25_mobility_pre.json',
    'SEIVR_Q simulation with Mobility Network (Pre)',
))

validations.append(lambda: run_validation_lab(
    MonitoredSEIVRQ(),
    mng_pre, 
    {
        **params_mobility_pre, 
        **v_params_seivr,
        **v_params_monitor,
        SEIVRQ.P_QUARANTINE: 0.5
    },
    'v_seivrq_50_mobility_pre.json',
    'SEIVR_Q simulation with Mobility Network (Pre)',
))

validations.append(lambda: run_validation_lab(
    MonitoredSEIVRQ(),
    mng_pre, 
    {
        **params_mobility_pre, 
        **v_params_seivr,
        **v_params_monitor,
        SEIVRQ.P_QUARANTINE: 0.75
    },
    'v_seivrq_75_mobility_pre.json',
    'SEIVR_Q simulation with Mobility Network (Pre)',
))

#### Post

In [45]:
validations.append(lambda: run_validation_lab(
    MonitoredSEIVRQ(),
    mng_post, 
    {
        **params_mobility_post, 
        **v_params_seivr,
        **v_params_monitor,
        SEIVRQ.P_QUARANTINE: 0.25
    },
    'v_seivrq_25_mobility_post.json',
    'SEIVR_Q simulation with Mobility Network (Post)',
))

validations.append(lambda: run_validation_lab(
    MonitoredSEIVRQ(),
    mng_post, 
    {
        **params_mobility_post, 
        **v_params_seivr,
        **v_params_monitor,
        SEIVRQ.P_QUARANTINE: 0.5
    },
    'v_seivrq_50_mobility_post.json',
    'SEIVR_Q simulation with Mobility Network (Post)',
))

validations.append(lambda: run_validation_lab(
    MonitoredSEIVRQ(),
    mng_post, 
    {
        **params_mobility_post, 
        **v_params_seivr,
        **v_params_monitor,
        SEIVRQ.P_QUARANTINE: 0.75
    },
    'v_seivrq_75_mobility_post.json',
    'SEIVR_Q simulation with Mobility Network (Post)',
))

### Run

In [46]:
print("Start time %s" % dt.now())


for v in validations:
    v()


print("End time %s" % dt.now())

Start time 2021-07-30 10:13:44.076084
v_seivrq_25_mobility_pre.json
Experiment v_seivrq_25_mobility_pre.json done: 2021-07-30 10:20:52.628573
v_seivrq_50_mobility_pre.json
Experiment v_seivrq_50_mobility_pre.json done: 2021-07-30 10:27:57.855718
v_seivrq_75_mobility_pre.json
Experiment v_seivrq_75_mobility_pre.json done: 2021-07-30 10:34:58.882682
v_seivrq_25_mobility_post.json
Experiment v_seivrq_25_mobility_post.json done: 2021-07-30 10:40:13.315112
v_seivrq_50_mobility_post.json
Experiment v_seivrq_50_mobility_post.json done: 2021-07-30 10:45:27.119916
v_seivrq_75_mobility_post.json
Experiment v_seivrq_75_mobility_post.json done: 2021-07-30 10:50:37.451036
End time 2021-07-30 10:50:38.669124


## Transform to app-data layout (validation)

Transform the validation results into the required app-data format and store
in data repo

In [3]:
from lib.experiments.utils.create_app_data import main_custom_files
from lib.experiments.utils.simulation_files import MODEL, NAME, NETWORK, MN_PRE, \
    PLC_PRE, DIST_PRE, MN_POST, PLC_POST, DIST_POST

validation_path = 'validation'

In [ ]:
seir_files = [
    {MODEL: 'SEIR', NETWORK: MN_PRE, NAME: 'v_seir_mobility_pre'},
    {MODEL: 'SEIR', NETWORK: PLC_PRE, NAME: 'v_seir_plc_pre'},
    {MODEL: 'SEIR', NETWORK: DIST_PRE, NAME: 'v_seir_distanced_pre'},
    
    {MODEL: 'SEIR', NETWORK: MN_POST, NAME: 'v_seir_mobility_post'},
    {MODEL: 'SEIR', NETWORK: PLC_POST, NAME: 'v_seir_plc_post'},
    {MODEL: 'SEIR', NETWORK: DIST_POST, NAME: 'v_seir_distanced_post'},

    {MODEL: 'SEIR_Q', NETWORK: MN_PRE, NAME: 'v_seirq_25_mobility_pre'},
    {MODEL: 'SEIR_Q', NETWORK: MN_PRE, NAME: 'v_seirq_50_mobility_pre'},
    {MODEL: 'SEIR_Q', NETWORK: MN_PRE, NAME: 'v_seirq_75_mobility_pre'},
    
    {MODEL: 'SEIR_Q', NETWORK: MN_POST, NAME: 'v_seirq_25_mobility_post'},
    {MODEL: 'SEIR_Q', NETWORK: MN_POST, NAME: 'v_seirq_50_mobility_post'},
    {MODEL: 'SEIR_Q', NETWORK: MN_POST, NAME: 'v_seirq_75_mobility_post'},
]

main_custom_files(seir_files, validation_path, validation_path)

In [ ]:
seivr_files = [
    {MODEL: 'SEIVR', NETWORK: MN_PRE, NAME: 'v_seivr_mobility_pre'},
    {MODEL: 'SEIVR', NETWORK: MN_POST, NAME: 'v_seivr_mobility_post'},

    {MODEL: 'SEIVR_Q', NETWORK: MN_PRE, NAME: 'v_seivrq_25_mobility_pre'},
    {MODEL: 'SEIVR_Q', NETWORK: MN_PRE, NAME: 'v_seivrq_50_mobility_pre'},
    {MODEL: 'SEIVR_Q', NETWORK: MN_PRE, NAME: 'v_seivrq_75_mobility_pre'},
    
    {MODEL: 'SEIVR_Q', NETWORK: MN_POST, NAME: 'v_seivrq_25_mobility_post'},
    {MODEL: 'SEIVR_Q', NETWORK: MN_POST, NAME: 'v_seivrq_50_mobility_post'},
    {MODEL: 'SEIVR_Q', NETWORK: MN_POST, NAME: 'v_seivrq_75_mobility_post'},
]

main_custom_files(seivr_files, validation_path, validation_path)